In [1]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

### Metadata study active cards and number of registers per day

In [2]:
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 10, 30)
dates = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end_date - start_date).days + 1)]

unique_ids = []
num_registers = []
for date in tqdm(dates):
    usersDf =  pd.read_csv(os.path.join(os.getcwd(), "intermediate", f"{date}_filtered_data.txt"), sep=';')
    unique_ids.append(len(usersDf.cardID.unique()))
    num_registers.append(usersDf.shape[0])


  0%|          | 0/303 [00:00<?, ?it/s]

100%|██████████| 303/303 [03:23<00:00,  1.49it/s]


In [3]:
meta_df = pd.DataFrame()
meta_df["dates"] = dates
meta_df["unique_cards"] = unique_ids
meta_df["num_registers"] = num_registers

In [4]:
meta_df["dates"] = pd.to_datetime(meta_df["dates"], format="%Y-%m-%d")

In [5]:
import plotly.express as px

fig = px.line(meta_df, x="dates", y="unique_cards", title='Senior contract active smartcards per day',
              labels={'x': "", 'y':'Number of smartcards'})
fig.update_layout(xaxis=dict(showgrid=True),
              yaxis=dict(showgrid=True, gridcolor='lightgrey'),
              plot_bgcolor='white',
              xaxis_title="", yaxis_title="Active cards per day",
              title_x=0.5, title_y=0.85,
)
fig.show()

In [7]:
import plotly.express as px

fig = px.line(meta_df, x="dates", y="num_registers", title='Senior smartcards transactions per day',
              labels={'x': "", 'y':'Number of transactions'})
fig.update_layout(xaxis=dict(showgrid=True),
              yaxis=dict(showgrid=True, gridcolor='lightgrey'),
              plot_bgcolor='white',
              xaxis_title="", yaxis_title="Active cards per day",
              title_x=0.5, title_y=0.85,
)
fig.show()
# fig.write_image(}"total_transactions_per_day.png")

In [46]:
meta_df.to_csv("meta_df.csv",index=False, sep="|")

### Selection of top #10 od-pairs by total number of trips

In [9]:
# load dataset and select just one od pair
dataset = pd.read_csv(os.path.join(os.getcwd(), "output", "final_odm_with_weather.csv"), dtype="str")
dataset = dataset.astype({'trips': float, 'temperature_2m (°C)': float, 'rain (mm)': float, 'snowfall (cm)': float})
dataset["od"] = dataset["origen"]+ "|" + dataset["destino"]

# Format
dataset["fecha"] = pd.to_datetime(dataset["fecha"], format="%Y-%m-%d %H:%M:%S")
dataset = dataset.drop(columns=["origen","destino"]).set_index("fecha")

In [19]:
# Select top 10 od pairs
dataset.groupby("od", as_index=False).agg({"trips":"sum"}).sort_values(by="trips", ascending=False).head(10).reset_index(drop=True).od

0    28079|28079
1    28074|28074
2    28005|28005
3    28092|28092
4    28065|28065
5    28058|28058
6    28007|28007
7    28148|28148
8    28079|28007
9    28079|28092
Name: od, dtype: object